In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('fatal-police-shootings-data.csv')

In [2]:
grouped = df.groupby(['race', 'signs_of_mental_illness']).size().reset_index(name='counts')
print(grouped)


   race  signs_of_mental_illness  counts
0     A                    False      69
1     A                     True      24
2     B                    False    1114
3     B                     True     184
4     H                    False     745
5     H                     True     157
6     N                    False      64
7     N                     True      14
8     O                    False      37
9     O                     True      11
10    W                    False    1763
11    W                     True     713


In [3]:
grouped_df = df.groupby(['race', 'signs_of_mental_illness']).size().reset_index(name='counts')

def odsetek(x):
    if x == True: 
        return 1
    else:
        return 0

grouped_df['odsetek'] = grouped_df['counts'] / grouped_df['counts'].sum()
grouped_df['odsetek_flag'] = grouped_df['signs_of_mental_illness'].apply(odsetek)
grouped_df['odsetek'] = grouped_df['odsetek'].apply(lambda x: '{:.2%}'.format(x))

grouped_df



,race,signs_of_mental_illness,counts,odsetek,odsetek_flag
0,A,False,69,1.41%,0
1,A,True,24,0.49%,1
2,B,False,1114,22.76%,0
3,B,True,184,3.76%,1
4,H,False,745,15.22%,0
5,H,True,157,3.21%,1
6,N,False,64,1.31%,0
7,N,True,14,0.29%,1
8,O,False,37,0.76%,0
9,O,True,11,0.22%,1


In [4]:
import matplotlib.pyplot as plt

df['day'] = pd.Categorical(df['day'], ordered=True)
day_counts = df['day'].value_counts().sort_index()

ax = day_counts.plot(kind='bar', color='red')
ax.set_title('Zliczenie interwencji według dnia tygodnia')
ax.set_xlabel('Dzień tygodnia')
ax.set_ylabel('Liczba interwencji')

plt.tight_layout()
plt.show()


KeyError: 'day'

In [ ]:

population_df = pd.read_excel('populacja_stany.xlsx')

fatal_df = pd.read_excel('stany.xlsx')


fatal_df['Census population, April 1, 2010'] = pd.to_numeric(fatal_df['Census population, April 1, 2010'], errors='coerce')
population_df['Population'] = pd.to_numeric(population_df['Population'], errors='coerce')

df['Incydenty śmiertelne w USA'] = (fatal_df['Census population, April 1, 2010'] / population_df['Population']) * 1000
df['Incydenty śmiertelne w USA'] = df['Incydenty śmiertelne w USA'].apply(lambda x: '{:.2%}'.format(x) if pd.notnull(x) else x)
df


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,Incydenty śmiertelne w USA
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,NaN
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,NaN
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,NaN
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,NaN
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5411,5921,William Slyter,2020-06-13,shot,gun,22.0,M,W,Kansas City,MO,False,other,Other,False,NaN
5412,5922,TK TK,2020-06-13,shot,undetermined,NaN,M,NaN,San Bernardino,CA,False,attack,Not fleeing,False,NaN
5413,5924,Nicholas Hirsh,2020-06-15,shot,gun,31.0,M,W,Lawrence,KS,False,attack,Car,False,NaN
5414,5926,TK TK,2020-06-16,shot,gun,24.0,M,NaN,Beach Park,IL,False,attack,Not fleeing,False,NaN
